In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111
ERROR: No matching distribution found for torch==1.8.1+cu111


In [2]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re

In [3]:
file_path = "cleaned_dataset_10percent.csv"
dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
dataset.head()

,ID,interview_date,country,NPS,translated_comment,customer_type,comment_length,translated_comment_preprocessed
0,213278,2022-09-10,Czech,10.0,&;&;&;Processing the request &;&;&;,promoters,59,processing request
1,174512,2021-02-10,Sweden,5.0,E.on Pay more,detractors,13,eon pay
2,48192,2021-05-20,Netherlands,8.0,am satisfied with essence,passives,25,satisfied essence
3,118266,2021-08-21,Sweden,8.0,Can't answer the question.,passives,30,cant answer question
4,160338,2021-08-06,Romania,8.0,"hurry up, until I made the appointment, why di...",passives,68,hurry made appointment didnt go directly


In [4]:
# Function for sentiment scores
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [5]:
# Utilizing pretrained model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

sentiment_scores = []

# Iterate through each row in the DataFrame
for index, row in dataset.iterrows():
    # Calculate sentiment score and append to the list
    sentiment_scores.append(sentiment_score(row['translated_comment'][:1024]))

In [6]:
# Assigning the sentiment scores to a new column
dataset['bert_sentiment'] = sentiment_scores

In [8]:
# Saving dataset
dataset.to_csv("tenpercent_bert.csv", index=False)